# LangGraph 워크플로우 설계

## 상태 클래스(`state.py`) 정의

In [ ]:
from typing import Literal, TypedDict, Annotated

def reduce_list(left: list, right: list) -> list:
    """두 리스트를 합칩니다."""
    return left + right

# 애플리케이션 상태 모델 정의
# 모든 필드를 선택적(total=False)으로 관리 => 상태 관리에 적합한 방식
class AppState(TypedDict, total=False):
    """
    애플리케이션의 전체 상태를 관리하는 중앙 저장소.
    Annotated를 사용하여 각 필드에 대한 설명을 타입 힌트에 포함합니다.
    """
    
    # --- 공통 및 초기 필드 ---
    user_input: Annotated[str, "사용자의 현재 입력값"]
    chat_history: Annotated[list[tuple[str, str]], "UI용 대화 기록 리스트", reduce_list]
    role: Annotated[Literal["student", "professor", "unknown"], "현재 사용자의 역할"]


## 노드 함수 정의

### 응시자 검증 노드 함수 (`applicant.py`)

In [ ]:
def applicant_validator(state: AppState) -> AppState:
    """응시자 정보 검증 & 중복 응시 확인"""
    pass

### 퀴즈 출제 노드 함수 (`quiz.py`)

In [ ]:
def quiz_setter(state: AppState) -> AppState:
    """퀴즈 문항 설정"""
    pass

def continue_quiz_condition(state: AppState) -> str:
    """퀴즈 계속/채점 분기 키 일치"""
    questions = state.get("questions", [])
    quiz_index = state.get("quiz_index", 0)
    if quiz_index < len(questions):
        return "continue_quiz"
    else:
        return "grade_quiz"

def quiz_popper(state: AppState) -> AppState:
    """현재 문제 출력"""
    pass

def answer_collector(state: AppState) -> AppState:
    """답변 수집 및 다음 인덱스"""
    pass

def grading_prompter(state: AppState) -> AppState:
    """채점 프롬프트 생성"""
    pass

def grade_reporter(state: AppState) -> AppState:
    """LLM 채점 → FinalReport 파싱"""
    pass

def report_formatter(state: AppState) -> AppState:
    """FinalReport → 사람이 읽을 수 있는 텍스트"""
    pass


### 채점 결과 리포트 노드 함수 (`report.py`)

In [ ]:
def grade_report_saver(state: AppState) -> AppState:
    """채점 결과 DB 저장 (숫자 집계로 저장)"""
    pass

def report_request_parser(state: AppState) -> AppState:
    """리포트 요청 파싱"""
    pass

def report_generater(state: AppState) -> AppState:
    """요청 조건에 맞는 리포트 생성"""
    pass


### 사용자 역할에 따른 라우팅 처리 노드 함수 (`routing.py`)

In [ ]:
def entry_router(state: AppState) -> str:
    """역할 분류 및 진입점 라우터"""
    pass

def entry_helper(state: AppState) -> AppState:
    """알 수 없는 역할에 대한 도움말"""
    pass

## 노드 함수 연결 및 그래프 생성 (`graph.py`)

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AppState)

# 노드 추가
graph.add_node("entry_helper", entry_helper)
graph.add_node("applicant_validator", applicant_validator)
graph.add_node("quiz_setter", quiz_setter)
graph.add_node("quiz_popper", quiz_popper)
graph.add_node("answer_collector", answer_collector)
graph.add_node("grading_prompter", grading_prompter)
graph.add_node("grade_reporter", grade_reporter)
graph.add_node("grade_report_saver", grade_report_saver)
graph.add_node("report_formatter", report_formatter)
graph.add_node("report_request_parser", report_request_parser)
graph.add_node("report_generater", report_generater)

# 조건부 엔트리
graph.set_conditional_entry_point(
    entry_router,
    {
        "quiz_entry": "quiz_setter",
        "answer_entry": "answer_collector",
        "student_entry": "applicant_validator",
        "professor_entry": "report_request_parser",
        "unknown_entry": "entry_helper",
    },
)

# 엣지
graph.add_edge("quiz_setter", "quiz_popper")
graph.add_edge("quiz_popper", END)  # 문제 출력 후 턴 종료
graph.add_edge("entry_helper", END)

graph.add_conditional_edges(
    "answer_collector",
    continue_quiz_condition,
    {"continue_quiz": "quiz_popper", "grade_quiz": "grading_prompter"},
)
graph.add_edge("grading_prompter", "grade_reporter")
graph.add_edge("grade_reporter", "grade_report_saver")
graph.add_edge("grade_report_saver", "report_formatter")
graph.add_edge("report_request_parser", "report_generater")
graph.add_edge("report_formatter", END)
graph.add_edge("report_generater", END)

workflow = graph.compile()


### 그래프 시각화

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(workflow)